problem

the data is bad, it has duplicate entries with tx hash and time

idea: find how often this occurs

if its not too many delete the entries 

or clean the data and change the times

In [1]:
import pandas as pd
from datetime import datetime

In [55]:
df_full = pd.read_csv("/Users/haaroony/OneDrive - Pometry Ltd/nft_andrea/Data_API.csv")

/var/folders/rm/nnz28pc96cv2xqzbwlwvwl2r0000gn/T/ipykernel_26651/1900226705.py:1: DtypeWarning: Columns (0,1,2,4,6,8,9,10,14,15,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv("/Users/haaroony/OneDrive - Pometry Ltd/nft_andrea/Data_API.csv")


In [49]:
drop_some_column = ["Unique_id_collection", "Permanent_link", "Description", "Name", "Image_url_4", "Image_url_3", "Image_url_2", "Image_url_1"]
df_dropped = df_full.drop(columns=drop_some_column)

In [3]:
df = df_full[['ID_token', 'Transaction_hash', 'Datetime_updated_seconds']]

In [5]:
s = "2019-11-30 23:59:33"
df['unix_time'] = df['Datetime_updated_seconds'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").timestamp())

/var/folders/rm/nnz28pc96cv2xqzbwlwvwl2r0000gn/T/ipykernel_26651/3612290062.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['unix_time'] = df['Datetime_updated_seconds'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").timestamp())


old=(1540436348,0xfbf193668516edb858e1ac8b95a562d2a8583418e9dd3cf523b66e6a7426e17c),
new=(1540436348,0x3cd1d9caa9d06e40aef1b052676c72b66d9de81d68dc0d0a0576fc0078a8dabd)

In [31]:
from collections import Counter

print(" NFTS: \n" + str(len(set(df['ID_token']))))

c=0
bad_nfts = set([])
for tid, gp_df in df.groupby(by=['ID_token']):
    c+=1
    if c % 200_000 == 0:
        print(c)
    pairs = list(zip(gp_df['Transaction_hash'],gp_df['unix_time']))
    counted = Counter(pairs)
    counted_total = set(counted.values())
    if max(counted_total) >= 2:
        print("Found a bad nft " + str(tid))
        bad_nfts.add(tid)

 NFTS: 
4746857
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
2000000
2200000
2400000
2600000
2800000
3000000
3200000
3400000
3600000
3800000
4000000
4200000
4400000
Found a bad nft 520975
Found a bad nft 597444
Found a bad nft 614950
Found a bad nft 616766
Found a bad nft 630892
Found a bad nft 640086
Found a bad nft 6877
Found a bad nft 712882
Found a bad nft 712913
4600000
Found a bad nft 856183
Found a bad nft 9103
Found a bad nft 9672
Found a bad nft 992744


java.lang.Error: Mismatched duplicate element: old=(1540436348,0xfbf193668516edb858e1ac8b95a562d2a8583418e9dd3cf523b66e6a7426e17c), new=(1540436348,0x3cd1d9caa9d06e40aef1b052676c72b66d9de81d68dc0d0a0576fc0078a8dabd
d68dc0d0a0576fc0078a8dabd)
        at com.raphtory.internals.storage.pojograph.OrderedBuffer$OrderedAppend.sortedAppend(OrderedBuffer.scala:34)
        at com.raphtory.internals.storage.pojograph.entities.internal.MutableProperty.update(MutableProperty.scala:27)
        at com.raphtory.internals.storage.pojograph.entities.internal.PojoEntity.$plus(PojoEntity.scala:57)
        at com.raphtory.internals.storage.pojograph.PojoBasedPartition.$anonfun$addProperties$1(PojoBasedPartition.scala:33)
        at com.raphtory.internals.storage.pojograph.PojoBasedPartition.$anonfun$addProperties$1$adapted(PojoBasedPartition.scala:32)
        at scala.collection.IterableOnceOps.foreach(IterableOnce.scala:563)
        at scala.collection.IterableOnceOps.foreach$(IterableOnce.scala:561)
        at scala.collection.AbstractIterable.foreach(Iterable.scala:926)
        at com.raphtory.internals.storage.pojograph.PojoBasedPartition.addProperties(PojoBasedPartition.scala:32)
        at com.raphtory.internals.storage.pojograph.PojoBasedPartition.addEdge(PojoBasedPartition.scala:231)
        at com.raphtory.internals.components.partition.BatchWriter.processEdgeAdd(BatchWriter.scala:58)
        at com.raphtory.internals.components.partition.BatchWriter.handleMessage(BatchWriter.scala:29)
        at com.raphtory.api.input.GraphBuilder.handleEdgeAdd(GraphBuilder.scala:267)
        at com.raphtory.api.input.GraphBuilder.addEdge(GraphBuilder.scala:239)
        at com.raphtory.api.input.GraphBuilder.addEdge$(GraphBuilder.scala:231)
        at com.raphtory.examples.nft.graphbuilder.NFTGraphBuilder.addEdge(NFTGraphBuilder.scala:12)
        at com.raphtory.examples.nft.graphbuilder.NFTGraphBuilder.parseTuple(NFTGraphBuilder.scala:96)
        at com.raphtory.examples.nft.graphbuilder.NFTGraphBuilder.parseTuple(NFTGraphBuilder.scala:12)
        at com.raphtory.internals.components.partition.LocalBatchHandler.$anonfun$runIngestion$1(LocalBatchHandler.scala:54)

In [35]:
len(bad_nfts)

13

In [36]:
len(bad_nfts)/4746857*100

0.00027386542295249256

In [52]:
df_dropped.to_csv("/Users/haaroony/OneDrive - Pometry Ltd/nft_andrea/Data_API_cleaned.csv", index=False)

In [53]:
len(df_dropped)

6071027

In [63]:
df_dropped

Smart_contract              5109417
ID_token                    6071027
Transaction_hash            2885387
Seller_address              6071027
Seller_username             1093254
Buyer_address               6071027
Buyer_username               892863
Price_Crypto                6071027
Crypto                      6071027
Price_USD                   6062744
Collection                  6071027
Market                      6071027
Datetime_updated            6071027
Datetime_updated_seconds    6071027
Collection_cleaned          6071027
Category                    6071027
dtype: int64

In [65]:
set(df_dropped['Crypto'])

{'0xBTC',
 '1MT',
 '2XDN',
 'ABST',
 'AMPL',
 'ANRX',
 'ARCONA',
 'ART',
 'ASLT',
 'ATRI',
 'AVRT',
 'B0T',
 'BAEPAY',
 'BASED',
 'BAT',
 'BLVD',
 'BON',
 'BONDLY',
 'BONES',
 'BOOB',
 'BORG',
 'BPC',
 'BUDGET',
 'BUDZ',
 'BZN',
 'CAMEL',
 'CGG',
 'CHERRY',
 'CHONK',
 'COIN',
 'COVAL',
 'CPT',
 'CRED',
 'CUBE',
 'CURIO',
 'DAI',
 'DAPPT',
 'DDIM',
 'DENA',
 'DGX',
 'DHC',
 'DOOM',
 'DUST',
 'EBB',
 'ECAT',
 'ECTO',
 'EGGS',
 'ELAND',
 'ELET',
 'EMONT',
 'ENJ',
 'EPIC',
 'ETH',
 'FIRST',
 'FRFY',
 'FTHR',
 'FTM',
 'FUD',
 'GALA',
 'GCASH',
 'GEM',
 'GMX',
 'GOKU',
 'GOOSE',
 'GOU',
 'GPL',
 'GUSD',
 'HIVED',
 'HOUR',
 'HUE',
 'IMP',
 'INI',
 'INK',
 'JBG',
 'JGN',
 'KAP',
 'KEK',
 'KEK-DEP',
 'KING',
 'KIWI',
 'KLTR',
 'KOI',
 'LAR',
 'LESS',
 'LINK',
 'LIT',
 'LOAD',
 'MANA',
 'MATIC',
 'MBC',
 'MCX',
 'MEME',
 'MGDv2',
 'MKR',
 'MM',
 'MNFT',
 'MORK',
 'MX',
 'NDR',
 'NEWS',
 'NUGS',
 'NVT',
 'OLDWAXIE',
 'OLDWZX',
 'PGU',
 'PIXEL',
 'PIXIE',
 'PLAY',
 'PMON',
 'PPDEX',
 'PRIME',
 'PX

In [66]:
df_dropped_eth_only = df_dropped[df_dropped['Crypto']=='ETH']

In [68]:
df_dropped_eth_only.to_csv("/Users/haaroony/OneDrive - Pometry Ltd/nft_andrea/Data_API_cleaned_ETH_only.csv", index=False)

In [69]:
max(df_dropped_eth_only['Datetime_updated_seconds'])

'2021-04-27 23:59:51'

In [60]:
for i, x in enumerate(df_dropped.columns):
    print(i, x)

0 Smart_contract
1 ID_token
2 Transaction_hash
3 Seller_address
4 Seller_username
5 Buyer_address
6 Buyer_username
7 Price_Crypto
8 Crypto
9 Price_USD
10 Collection
11 Market
12 Datetime_updated
13 Datetime_updated_seconds
14 Collection_cleaned
15 Category


In [50]:
max)df_clean_dropped_nfts)

NameError: name 'df_clean_dropped_nfts' is not defined

## unclean data

In [43]:
unclean_df = pd.read_csv("/Users/haaroony/OneDrive - Pometry Ltd/nft_andrea/bad_nft.csv")

In [44]:
unclean_df

,Smart_contract,ID_token,Transaction_hash,Seller_address,Seller_username,Buyer_address,Buyer_username,Image_url_1,Image_url_2,Image_url_3,...,Name,Description,Collection,Market,Datetime_updated,Datetime_updated_seconds,Permanent_link,Unique_id_collection,Collection_cleaned,Category
0,0x06012c8cf97bead5deae237070f9587f8e7a266d,539565,0xb0dd3d7199c3df1e81f99f304a1bfc834373684468aa...,0x4886404ecfb86b759b27867afb0f64172eea1c53,PsyDuck,0xefe92c9018c8fd8bf52a2107f6177078a291353f,NaN,https://storage.opensea.io/0x06012c8cf97bead5d...,https://img.cryptokitties.co/0x06012c8cf97bead...,https://storage.opensea.io/0x06012c8cf97bead5d...,...,💜💙 Wuvme Chronic Bobtail,Bonjour! I'm 💜💙 Wuvme Chronic Bobtail. I work ...,Cryptokitties,OpenSea,2018-02-24 00:00:00,2018-02-24 20:21:59,https://opensea.io/assets/0x06012c8cf97bead5de...,"('Cryptokitties', '539565')",Cryptokittie,Art


In [47]:
unclean_df.drop(columns=drop_some_column)

,Smart_contract,ID_token,Transaction_hash,Seller_address,Seller_username,Buyer_address,Buyer_username,Price_Crypto,Crypto,Price_USD,Collection,Market,Datetime_updated,Datetime_updated_seconds,Collection_cleaned,Category
0,0x06012c8cf97bead5deae237070f9587f8e7a266d,539565,0xb0dd3d7199c3df1e81f99f304a1bfc834373684468aa...,0x4886404ecfb86b759b27867afb0f64172eea1c53,PsyDuck,0xefe92c9018c8fd8bf52a2107f6177078a291353f,NaN,0.004,ETH,3.37072,Cryptokitties,OpenSea,2018-02-24 00:00:00,2018-02-24 20:21:59,Cryptokittie,Art
